<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex12notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex12notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML)


----
## 次元削減(1) 主成分分析 - 前編
----




---
### 次元削減とは





#### ［復習］次元削減とは

**次元削減**(次元圧縮とも，dimensionality reduction)は，その名の通り，データの「次元数」(dimensionality)を減らす手法です．
$D$個の特徴量から成る $D$ 次元ベクトル $\mathbf{x}$ がデータとして与えられた場合，これを $H$ 次元（$H<D$）のベクトル $\mathbf{y}$ に変換します．
変換後のデータは元データよりも少数の特徴で表されているため，その後の分析がより簡単になります．
また，データを処理する際に必要な記憶容量や計算量の削減にもつながります．

例1: 学生たちの20科目の成績を表すデータ（$D=20$）を次元削減．

例えば，元データの特徴をよくとらえた3次元のデータに変換できたならば...．3つの特徴量の値によって一人ひとりの学生の傾向を分析することができる．次元を減らすことで分析や可視化が容易になる．

例2: 様々なひとの顔画像（幅100画素高さ100画素のRGBカラー画像，$D = 30000$）を次元削減．

元画像の本質的な特徴をなるべく保ったままで30次元のデータに変換できたならば...．ネットワーク越しに顔画像のデータを送信するとして，3万個の画素値を送るかわりに，30個の数値を送るだけで済ませられるかも？


#### 線形変換による次元削減

上記の通り，次元削減の問題は，データとして与えられる $D$ 次元ベクトル $\mathbf{x}\in {\cal R}^{D}$ を何らかの方法で $H$ 次元（$H<D$）ベクトルへと変換することです．
その方法として最も簡単なのは，線形変換（一次変換）を用いることです．

ベクトルを列ベクトルとして表すことにして，上記の $\mathbf{x}$ を

$$
\mathbf{x} = \begin{pmatrix}
x_1\\ x_2 \\ \vdots \\ x_D
\end{pmatrix}
$$

のように $D\times 1$ 行列として表します．このとき，適当な $H\times D$ 行列 $W$ を用意すれば，$\mathbf{y} = W\mathbf{x}$ という変換によって $H\times 1$行列すなわち$H$次元ベクトルを得ることができます．このように線形変換によって次元削減を実現する場合，この行列 $W$ を問題に応じて決めてやればよいことになります．つまり，この$W$が線形変換による次元削減のモデルパラメータです．

線形変換による次元削減は，次元削減の方法としては単純なものですが，データ分析や機械学習の分野で幅広く用いられています．
以下では，その方法の代表例として，「主成分分析」を取り上げます．
実際のデータ分析・機械学習の問題では，より高度な次元削減を実現するために非線形変換を利用したもっと複雑な方法も用いられます．

ここで，線形変換のことを理解しやすくするために，少し補足をしておきます．
上記の行列 $W$ の要素を
$$
W = \begin{pmatrix}
w_{1,1} & w_{1,2} & \dots & w_{1,D}\\
w_{2,1} & w_{2,2} & \dots & w_{2,D}\\
w_{H,1} & w_{H,2} & \dots & w_{H,D}
\end{pmatrix}
$$
と表して，
その各行（列ではありません）の要素をならべて次のような $D\times 1$ 行列をつくります．
$$
\mathbf{w}_h = \begin{pmatrix}
w_{h,1}\\
w_{h,2}\\
\vdots\\
w_{h,D}
\end{pmatrix} \quad (h = 1, 2, \ldots, H)
$$
つまり，
$$
W = \begin{pmatrix}
\mathbf{w}_{1}^{\top}\\
\mathbf{w}_{2}^{\top}\\
\vdots\\
\mathbf{w}_{H}^{\top}
\end{pmatrix}
$$
です．このとき，
$$
W\mathbf{x} = \begin{pmatrix}
\mathbf{w}_{1}^{\top}\\
\mathbf{w}_{2}^{\top}\\
\vdots\\
\mathbf{w}_{H}^{\top}
\end{pmatrix} \mathbf{x}
= \begin{pmatrix}
\mathbf{w}_{1}^{\top}\mathbf{x}\\
\mathbf{w}_{2}^{\top}\mathbf{x}\\
\vdots\\
\mathbf{w}_{H}^{\top}\mathbf{x}
\end{pmatrix}
$$
となりますので，
$$
\mathbf{y} = \begin{pmatrix}
y_{1}\\
y_{2}\\
\vdots\\
y_{H}
\end{pmatrix} = \begin{pmatrix}
\mathbf{w}_{1}^{\top}\mathbf{x}\\
\mathbf{w}_{2}^{\top}\mathbf{x}\\
\vdots\\
\mathbf{w}_{H}^{\top}\mathbf{x}
\end{pmatrix}
$$
と書けます．したがって，変換後のベクトルの $h$ 番目の要素 $y_h$ は
$$
y_h = \mathbf{w}_{h}^{\top}\mathbf{x}\quad (h = 1, 2, \ldots, H)
$$
となります．この式の右辺は，$D$次元ベクトル $\mathbf{w}_h$ と $\mathbf{x}$ の内積に等しいですので，次のように表記することもできます．
$$
y_h = \mathbf{w}_h\cdot\mathbf{x} \quad (h = 1, 2, \ldots, H)
$$
すなわち，$H\times D$ 行列 $W$ による $\mathbf{y} = W\mathbf{x}$ という変換は，$H$本の$D$次元ベクトル $\mathbf{w}_h$ ($h = 1, 2, \ldots, H$)を用意して，そのそれぞれと $\mathbf{x}$ との内積を求める計算をしていることに相当します．

####［余談だよん］

上記では，データひとつを $D\times 1$ 行列として表記していますので，要素が縦に並んでいます．
しかし，コンピュータでデータを扱う際は，数値を横に並べて，1行が1つのデータという形式で表すことがよくあります．
その場合，複数のデータをまとめて扱う際に，それを表す配列・行列が数式で想定しているものと縦横逆になっている，つまり転置になっていたりしますので，注意が必要です．

ちなみに，データひとつを $1 \times D$ 行列として扱うことにした場合，上記の変換は
$$
\mathbf{y} = \mathbf{x}W^{\top}
$$
と書けます．

---
### 主成分分析（前編）

データ分析・多変量解析の方法としてよく知られた **主成分分析** (Principal Component Analysis, PCA)は，線形変換による次元削減の方法のひとつです．
ここでは，削減後の次元数が $1$ である場合，つまり，$D$ 次元のベクトルを1つの数値に変換する場合に限定して，主成分分析を導入します．

#### 主成分分析の考え方

主成分分析では，「変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）」次元削減を行います．
ここでは，元のデータの次元数 $D = 2$ の場合を例として，これを1次元に次元削減する場合を図に描きながら考えてみましょう．



下図の左は，とある2次元のデータを散布図に描いたものです．二つの破線はそれぞれ，横軸と縦軸の値の平均を表しています．
このデータの値を，それらの平均が $\mathbf{0}$ になるように平行移動させると，散布図は下図の右のようになります．
以降，このように平均が $\mathbf{0}$ になるように平行移動させたデータの集合を
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
と表すことにします．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/pcafig01.png">


このようなデータを1次元に次元削減する線形変換は，$D$次元ベクトル $\mathbf{u} \in {\cal R}^{D}$ をひとつ定めて，下図のように$D$次元ベクトル $\mathbf{x}$ を1つの実数値 $y$ に変換する操作と考えられます．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/pcafig02.png">

この図の左の赤い直線は，ベクトル $\mathbf{u}$ の向きを表しています．個々のデータ $\mathbf{x}$ に対して，「その点がこの赤い直線上に落とす影」を考えて，この直線上でのその影の位置を $y$ とおくと，右図に示すように
$$
y = \Vert\mathbf{x}\Vert\cos\theta
$$
となります．$\theta$ は $\mathbf{x}$ と $\mathbf{u}$ の成す角，$\Vert\mathbf{x}\Vert$ は $\mathbf{x}$ の大きさです．
$\mathbf{u}\cdot\mathbf{x} = \Vert\mathbf{u}\Vert\Vert\mathbf{x}\Vert\cos\theta$ であることから，上の式は，次のように書き換えられます．
$$
y = \frac{\mathbf{u}\cdot\mathbf{x}}{\Vert\mathbf{u}\Vert}
$$
与えられたデータに対して，ベクトル $\mathbf{u}$ を適当に定めてやれば，この式によって $D$ 次元のデータを1つの実数値に変換できます．

ここで，ベクトル $\mathbf{u}$  として，向きが同じで大きさのみ異なる二つの候補があったとしてみましょう．その場合，得られる $y$ の値はどちらを用いても同じとなります．つまり，ここで考えている変換では，ベクトル $\mathbf{u}$ の向きのみが重要ということになります．
そのため，ベクトル $\mathbf{u}$ の大きさは $1$ に限定して考えることができます．この場合，$\Vert\mathbf{u}\Vert = 1$ ですので，上の式はより簡単に
$$
y = \mathbf{u}\cdot\mathbf{x}
$$
と表せます．

以上より，大きさ$1$の$D$次元ベクトル $\mathbf{u}$ を適当に定めれば，$y_n = \mathbf{u}\cdot\mathbf{x}_n$ という式で1次元への次元削減ができることが分かりました．
では，$\mathbf{u}$ はどのように定めればよいでしょうか？

主成分分析の目的は，「変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）」することでした．$\mathbf{u}$の向きを変えると，変換後の値 $y_n$ の分散の大きさも変化します．
以下の図(a), (b), (c) は，3種類の $\mathbf{u}$ で2次元のデータを1次元に変換した様子と，変換後の値の分散を示しています．
右図は横軸に $y_n$ の値をとっており，'variance'の値が $y_n$ の分散です．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/pcafig03.png">

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/pcafig04.png">

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/pcafig05.png">

図を見ると，データの散らばりが大きい方向を $\mathbf{u}$ が向いたときに，変換後の分散が大きくなっていることがわかります．
直感的な説明になりますが，変換後の値だけを見て元のデータの様子を想像するなら，分散が大きい方が，個々の値の違いがわかりやすくてうれしいですね．

ここまで，データを1次元に次元削減する場合限定で，主成分分析の考え方を説明してきました．
次のセクションでは，このような主成分分析の問題をきちんと定式化して，その解がどのようなものかを説明します．

#### 主成分分析の問題の解（1次元にする場合）

---
［主成分分析の問題設定（1次元にする場合）］

$N$個の$D$次元ベクトルから成る学習データが与えられるとする．
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
このデータの平均は
$$
\frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_{n} = \mathbf{0}
$$
と仮定する（$\mathbf{0}$でない場合は，平均を求めてそれをデータから差し引いたものをあらためて学習データとすればよい）．
大きさ $1$ の $D$ 次元ベクトル $\mathbf{u}$ を用いて，次式によって $D$次元ベクトル $\mathbf{x}$ を1つの実数値 $y_n$ に変換することを考える．
$$
y_n = \mathbf{u}\cdot \mathbf{x}_n \quad (n = 1, 2, \ldots, N)
$$
ベクトル $\mathbf{u}$ を，大きさ1($\Vert\mathbf{u}\Vert = 1$)のベクトルの中で $y_n$の分散が最大となるように定めたい．

---

これが主成分分析の問題設定です．
この問題の解，つまり，$y_n$ の分散を最大にする $\mathbf{u}$ は，「**データ$\mathbf{x}_n$の分散共分散行列（注）の固有ベクトルのうち，最大の固有値に対応するもの**」となります（導出過程は後述）．
したがって，学習データから分散共分散行列を計算し，その固有値固有ベクトルを求めれば，主成分分析による次元削減を実現できます．

<span style="font-size: 75%">
※注: データ $\{ \mathbf{x}_n \}$ の分散共分散行列は
$$
V = \frac{1}{N}\sum_{n=1}^{N}(\mathbf{x}_{n} - \bar{\mathbf{x}})(\mathbf{x}_{n} - \bar{\mathbf{x}})^{\top}
$$

と定義されますが，ここでは詳しい説明は省略します．興味のあるひとは，データ分析・多変量解析・統計の書籍等で調べてみてね．
今の場合，データの平均が $\mathbf{0}$ ですので，$V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_{n}\mathbf{x}_{n}^{\top}$ となります．
</span>

#### ［発展］主成分分析の問題の解の導出（1次元にする場合）

上記の解の導出過程の概略を以下に示します．

$\mathbf{x}_n$ の平均が $\mathbf{0}$ なので，$y_n$ の平均も $0$ となります．そのため，$y_n$ の分散は
$$
\frac{1}{N}\sum_{n=1}^{N}y_n^2
$$
と表せます．この式は，次のように変形できます．
$$
\begin{aligned}
\frac{1}{N}\sum_{n=1}^{N}y_n^2 &= \frac{1}{N}\sum_{n=1}^{N} (\mathbf{u}\cdot\mathbf{x}_n)(\mathbf{u}\cdot\mathbf{x}_n) \\
&= \frac{1}{N}\sum_{n=1}^{N} \left(\mathbf{u}^{\top}\mathbf{x}_n\right) \left(\mathbf{u}^{\top}\mathbf{x}_n\right)^{\top}\\
&= \frac{1}{N}\sum_{n=1}^{N} \mathbf{u}^{\top}\mathbf{x}_n\mathbf{x}_n^{\top} \mathbf{u}\\
&= \mathbf{u}^{\top}\left( \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}\right) \mathbf{u} = \mathbf{u}^{\top}V\mathbf{u}
\end{aligned}
$$
ただし，
$$
V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}
$$
は，$\mathbf{x}_n$ の平均が $\mathbf{0}$ であるため，$\mathbf{x}_n$の分散共分散行列に一致します．



求めたいのは，$y_n$ の分散 $\mathbf{u}^\top V\mathbf{u}$ を最大にするベクトル $\mathbf{u}$ です．ただし，$\mathbf{u}$ には $\Vert\mathbf{u}\Vert = 1$ という条件があります．
このような制約条件付きの最適化問題を解く定番の手法は，「ラグランジュの未定乗数法」です（注）．
この問題の場合，ラグランジュ乗数を $\lambda$ として，
$$
L = \mathbf{u}^{\top}V\mathbf{u} - \lambda(\mathbf{u}^{\top}\mathbf{u} - 1)
$$
とおけば，$\frac{\partial L}{\partial \mathbf{u}} = \mathbf{0}$ を満たす $\mathbf{u}$ が解の候補となります．

<span style="font-size: 75%">
※注: 「ラグランジュの未定乗数法」は，大学初年次の微積分で学んでいる...かも．興味のあるひとは数学の参考書を調べてね．
</span>

ここで，
$$
\frac{\partial L}{\partial \mathbf{u}} = 2V\mathbf{u} - 2\lambda\mathbf{u}
$$
なので（注），解は
$$
V\mathbf{u} = \lambda\mathbf{u}
$$
を満たさねばなりません．この式より，解の候補は $V$ の単位固有ベクトルであることがわかります．この式を $L$ の式に代入すると，
$$
L = \lambda\mathbf{u}^\top\mathbf{u} - \lambda\cdot 0 = \lambda
$$
となりますので，この問題の解は，$V$の最大固有値に対する固有ベクトルであることがわかります．

<span style="font-size: 75%">
※注: $\frac{\partial L}{\partial \mathbf{u}}$ は，「$L$を $\mathbf{u}$ の各要素で偏微分したものをならべたベクトル」です．
</span>